# Camada Bronze:
---

## Tabela Produtos:

### Objetivo:

Entender a padronização e comportamento dos dados para criar a view produtos_bronze a partir da camada RAW.

### Fonte:

`workspace.default.produtos`

### Destino:

`workspace.olist_bronze.produtos_bronze`

In [0]:
%sql
-- Garantindo a existência do SCHEMA para armazenamento:

CREATE SCHEMA IF NOT EXISTS workspace.olist_bronze;

In [0]:
%sql
-- Verificando a existência da tabela:
SELECT * FROM workspace.default.produtos;

In [0]:
%sql
-- Verificando quantas linhas tem na tabela:
SELECT COUNT(*) AS LINHAS 
FROM workspace.default.produtos;

In [0]:
%sql
-- Verificando os tipos de dados:
DESC workspace.default.produtos;

Entre as colunas tipificadas como `BIGINT`, a que apresenta o maior número de algarismos é `product_weight_g`, com valores de até cinco dígitos. Nesse contexto, o uso de `BIGINT` é desnecessário e resulta em um consumo de memória maior do que o necessário. Assim, a conversão do tipo para `INT` é mais adequada, preservando a integridade dos dados e otimizando o uso de memória.

In [0]:
%sql
-- Verificando missing values em cada coluna:
SELECT
      SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS product_id_null,
      SUM(CASE WHEN product_category_name IS NULL THEN 1 ELSE 0 END) AS product_category_name_null,
      SUM(CASE WHEN product_name_lenght IS NULL THEN 1 ELSE 0 END) AS product_name_lenght_null,
      SUM(CASE WHEN product_description_lenght IS NULL THEN 1 ELSE 0 END) AS product_description_lenght_null,
      SUM(CASE WHEN product_photos_qty IS NULL THEN 1 ELSE 0 END) AS product_photos_qty_null,
      SUM(CASE WHEN product_weight_g IS NULL THEN 1 ELSE 0 END) AS product_weight_g_null,
      SUM(CASE WHEN product_length_cm IS NULL THEN 1 ELSE 0 END) AS product_length_cm_null,
      SUM(CASE WHEN product_height_cm IS NULL THEN 1 ELSE 0 END) AS product_height_cm_null,
      SUM(CASE WHEN product_width_cm IS NULL THEN 1 ELSE 0 END) AS product_width_cm_null
FROM workspace.default.produtos;

As colunas `product_category_name`, `product_name_lenght`, `product_description_lenght` e `product_photos_qty` apresentam a mesma quantidade de valores nulos, indicando que esses nulos provavelmente pertencem às mesmas linhas e estão associados a problemas de cadastro dos produtos.

In [0]:
%sql
SELECT 
        product_id,
        product_category_name,
        product_name_lenght,
        product_description_lenght,
        product_photos_qty
FROM workspace.default.produtos
WHERE product_category_name IS NULL;

Todos os demais valores nulos correspondem à mesma linha da coluna `product_category_name`. Dessa forma, é necessário avaliar o impacto da ausência de categoria desses produtos no volume total de pedidos antes de definir qualquer tratamento adicional.

In [0]:
%sql
-- Verificando missing values em cada coluna:
SELECT
      AVG(CASE WHEN product_id IS NULL THEN 100.00 ELSE 0 END) AS product_id_null_pct,
      AVG(CASE WHEN product_category_name IS NULL THEN 100.00 ELSE 0 END) AS product_category_name_null_pct,
      AVG(CASE WHEN product_name_lenght IS NULL THEN 100.00 ELSE 0 END) AS product_name_lenght_null_pct,
      AVG(CASE WHEN product_description_lenght IS NULL THEN 100.00 ELSE 0 END) AS product_description_lenght_null_pct,
      AVG(CASE WHEN product_photos_qty IS NULL THEN 100.00 ELSE 0 END) AS product_photos_qty_null_pct,
      AVG(CASE WHEN product_weight_g IS NULL THEN 100.00 ELSE 0 END) AS product_weight_g_null_pct,
      AVG(CASE WHEN product_length_cm IS NULL THEN 100.00 ELSE 0 END) AS product_length_cm_null_pct,
      AVG(CASE WHEN product_height_cm IS NULL THEN 100.00 ELSE 0 END) AS product_height_cm_null_pct,
      AVG(CASE WHEN product_width_cm IS NULL THEN 100.00 ELSE 0 END) AS product_width_cm_null_pct
FROM workspace.default.produtos;

Os produtos sem categoria representam menos de 2% do total de produtos.

In [0]:
%sql
SELECT 
    COUNT(DISTINCT p.order_id) AS total_pedidos_sem_categoria,
    ROUND(COUNT(DISTINCT p.order_id) * 100.0 / (SELECT COUNT(*) FROM workspace.default.pedidos), 2) AS porcentagem_do_total
FROM workspace.default.pedidos p
JOIN workspace.default.itens_pedido i ON p.order_id = i.order_id
JOIN workspace.default.produtos pr ON i.product_id = pr.product_id
WHERE pr.product_category_name IS NULL;

O volume de pedidos com produtos sem categoria representa menos de 2% do total.

In [0]:
%sql
-- Verificando valores inválidos em cada coluna:
SELECT 
      SUM(CASE WHEN product_id IS NOT NULL AND NOT product_id RLIKE '^[A-Za-z0-9]+$' THEN 1 ELSE 0 END) AS product_id_invalid,

      SUM(CASE WHEN product_category_name IS NOT NULL AND NOT product_category_name RLIKE '^[A-Za-z0-9_]+$' THEN 1 ELSE 0 END) AS product_category_name_invalid,

      SUM(CASE WHEN product_name_lenght IS NOT NULL AND product_name_lenght < 0 THEN 1 ELSE 0 END) AS product_name_lenght_invalid,

      SUM(CASE WHEN product_description_lenght IS NOT NULL AND product_description_lenght < 0 THEN 1 ELSE 0 END) AS product_description_lenght_invalid,

      SUM(CASE WHEN product_photos_qty IS NOT NULL AND product_photos_qty < 0 THEN 1 ELSE 0 END) AS product_photos_qty_invalid,

      SUM(CASE WHEN product_weight_g IS NOT NULL AND product_weight_g < 0 THEN 1 ELSE 0 END) AS product_weight_g_invalid,

      SUM(CASE WHEN product_length_cm IS NOT NULL AND product_length_cm < 0 THEN 1 ELSE 0 END) AS product_length_cm_invalid,

      SUM(CASE WHEN product_height_cm IS NOT NULL AND product_height_cm < 0 THEN 1 ELSE 0 END) AS product_height_cm_invalid,

      SUM(CASE WHEN product_width_cm IS NOT NULL AND product_width_cm < 0 THEN 1 ELSE 0 END) AS product_width_cm_invalid

FROM workspace.default.produtos;


Não há valores inválidos em nenhuma coluna.

In [0]:
%sql
CREATE OR REPLACE VIEW workspace.olist_bronze.produtos_bronze AS
SELECT *

FROM workspace.default.produtos;